In [1]:
import threading

In [18]:
def counter(n):
    print("start")
    th = threading.current_thread()
    print(th, th.name, th.is_alive(), th.daemon, th.ident, th.native_id)
    while n > 0:
        n -= 1

        
th = threading.Thread(
    target=counter,
    args=(100000000,),
    name="counter_example"
)
th.start()
print("before")
th.join()
print("after")


# counter(*args, **kwargs)

In [11]:
threading.active_count()

In [13]:
threading.current_thread(), threading.current_thread().name

In [14]:
threading.enumerate()

In [49]:
N = 100000
ITERS = 1000
# counter_calls = 0

def _counter(n):
#     global counter_calls
#     counter_calls += 1

    while n >= 0:
        n -= 1

        
def countdown(iters):
    for i in range(iters):        
        _counter(N)


In [44]:
%%time

countdown(ITERS)
print(counter_calls)

In [39]:
%%time

th = threading.Thread(
    target=countdown,
    args=(ITERS,),
    name="counter_example"
)
th.start()
th.join()

In [47]:
%%time

N_THREADS = 100


threads = [
    threading.Thread(
        target=countdown,
        args=(ITERS // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
    
print(counter_calls)

In [48]:
#GIL

In [50]:
from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(url, times):
    for _ in range(times):
        resp = fetch_url(url)

In [51]:
%%time

fetch_batch_urls(URL, N)

In [53]:
%%time

N_THREADS = 1


threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URL, N // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

In [55]:
%%time

N_THREADS = 10


threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URL, N // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

In [60]:
%%time

import time

global_count = [0]


def danger(iters):
    for _ in range(iters):
        c = global_count[0]
        c += 1
        time.sleep(0.0000000001)
        global_count[0] = c
        
        #global_count[0] += 1


threads = [
    threading.Thread(
        target=danger,
        args=(10,),
    )
    for _ in range(10000)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
    
print(global_count)

In [63]:
%%time

import time

global_count = [0]
lock = threading.Lock()


def danger_atomic(iters, lock):
    for _ in range(iters):
        lock.acquire()
        
        c = global_count[0]
        c += 1
        time.sleep(0.0000000001)
        global_count[0] = c
        
        lock.release()


threads = [
    threading.Thread(
        target=danger_atomic,
        args=(100, lock),
    )
    for _ in range(1000)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
    
print(global_count)

In [69]:
%%time

from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
N_THREADS = 10

sem = threading.Semaphore(2)


def fetch_url(url, sem):
    with sem:
        resp = urlopen(url)
    return resp


def fetch_batch_urls(url, times, sem):
    for _ in range(times):
        resp = fetch_url(url, sem)
        

threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URL, N // N_THREADS, sem),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

In [79]:
%%time

from urllib.request import urlopen
import queue


URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
N_THREADS = 10


sem = threading.Semaphore(10)
que = queue.Queue(20)


def fetch_url(sem, que):
    while True:
        try:
            url = que.get(timeout=1)
            if url is None:
                break
        except Exception:
            continue

        with sem:
            resp = urlopen(url)
            print("status", resp.code)
        

threads = [
    threading.Thread(
        target=fetch_url,
        args=(sem, que),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()


for _ in range(N):
    que.put(URL)

    
for _ in range(N_THREADS):
    que.put(None)

    
for th in threads:
    th.join()

In [82]:
import multiprocessing


class A:
    x = 42
    

def multy_func(dct):
    dct["data"] = A()
    dct["qwerty"] = 99


with multiprocessing.Manager() as manager:
    dct = manager.dict()
    
    print("before", dct)
    
    p = multiprocessing.Process(target=multy_func, args=(dct,))
    p.start()
    p.join()
    
    print("after", dct)